In [1]:
import dgl 
import torch as th

C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Using backend: pytorch


In [2]:
# 创建一个具有3种节点类型和3种边类型的异构图
# ('drug', 'interacts', 'drug') 括号的第一位和第三位分别表示实体，中间第二位表示二者的关系
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)
g.ntypes

['disease', 'drug', 'gene']

In [3]:
g.etypes

['interacts', 'interacts', 'treats']

In [4]:
g.canonical_etypes

[('drug', 'interacts', 'drug'),
 ('drug', 'interacts', 'gene'),
 ('drug', 'treats', 'disease')]

同构图和二分图只是一种特殊的异构图，它们只包括一种关系。

In [6]:
# 一个同构图
# dgl.heterograph({('node_type', 'edge_type', 'node_type'): (u, v)})

# 一个二分图
# dgl.heterograph({('source_type', 'edge_type', 'destination_type'): (u, v)})

与异构图相关联的 metagraph 就是图的模式。  
它指定节点集和节点之间的边的类型约束。   
metagraph 中的一个节点 u 对应于相关异构图中的一个节点类型。  
metagraph 中的边 (u,v) 表示在相关异构图中存在从 u 型节点到 v 型节点的边。

In [7]:
g

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])

In [8]:
g.metagraph().edges()

OutMultiEdgeDataView([('drug', 'drug'), ('drug', 'gene'), ('drug', 'disease')])

In [10]:
# 直接获取图中的所有节点会报错，需要指定获取哪种节点类型的节点
# 获取图中drug的节点
g.nodes('drug')

tensor([0, 1, 2])

In [11]:
# 获取图中所有节点的数量
print(g.num_nodes())

# 获取图中drug节点的数量
print(g.num_nodes('drug'))

10
3


获取特定的节点或边的特征

In [15]:
# 设置/获取"drug"类型的节点的"hv"特征
g.nodes['drug'].data['hv'] = th.ones(3, 1)
g.nodes['drug'].data['hv']

tensor([[1.],
        [1.],
        [1.]])

In [16]:
# 设置/获取"treats"类型的边的"he"特征
g.edges['treats'].data['he'] = th.zeros(1, 1)
g.edges['treats'].data['he']

tensor([[0.]])

如果图里只有一种节点或边类型，则不需要指定节点或边的类型。

In [17]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'is similar', 'drug'): (th.tensor([0, 1]), th.tensor([2, 3]))
})

g.ndata['hv'] = th.ones(4, 1)

从原图中创建子图

In [18]:
# 原图
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
    })

g.nodes['drug'].data['hv'] = th.ones(3, 1)

In [22]:
g

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])

In [19]:
# 创建子图，保留关系 ('drug', 'interacts', 'drug') 和 ('drug', 'treats', 'disease')
eg = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                                ('drug', 'treats', 'disease')])

In [20]:
eg

Graph(num_nodes={'disease': 3, 'drug': 3},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'disease', 'treats')])

In [21]:
# 相关的特征也会被拷贝
eg.nodes['drug'].data['hv']

tensor([[1.],
        [1.],
        [1.]])

#### **将异构图转换为同构图**

异构图为管理不同类型的节点和边及其相关特征提供了一个清晰的接口。这在以下情况下尤其有用:

1. 不同类型的节点和边的特征具有不同的数据类型或大小。

2. 用户希望对不同类型的节点和边应用不同的操作。

如果上述情况不适用，并且用户不希望在建模中区分节点和边的类型，则DGL允许使用 dgl.DGLGraph.to_homogeneous() API将异构图转换为同构图。 具体行为如下:

1. 用从0开始的连续整数重新标记所有类型的节点和边。

2. 对所有的节点和边合并用户指定的特征。

In [28]:
g = dgl.heterograph({
                   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
                   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
                    })

g.nodes['drug'].data['hv'] = th.zeros(3, 1)
g.nodes['disease'].data['hv'] = th.ones(3, 1)
g.edges['interacts'].data['he'] = th.zeros(2, 1)
g.edges['treats'].data['he'] = th.zeros(1, 2)

In [33]:
g

Graph(num_nodes={'disease': 3, 'drug': 3},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'disease', 'treats')])

In [29]:
# 默认情况下不进行特征合并
hg = dgl.to_homogeneous(g)
'hv' in hg.ndata

False

In [32]:
hg

Graph(num_nodes=6, num_edges=3,
      ndata_schemes={'hv': Scheme(shape=(1,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})

In [30]:
# 拷贝边的特征
# 对于要拷贝的特征，DGL假定不同类型的节点或边的需要合并的特征具有相同的大小和数据类型
hg = dgl.to_homogeneous(g, edata=['he'])

DGLError: Cannot concatenate column he with shape Scheme(shape=(2,), dtype=torch.float32) and shape Scheme(shape=(1,), dtype=torch.float32)

In [31]:
# 拷贝节点特征
hg = dgl.to_homogeneous(g, ndata=['hv'])
hg.ndata['hv']

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.]])

出于建模的目的，用户可能需要将一些关系合并，并对它们应用相同的操作。为了实现这一目的，可以先抽取异构图的边类型子图，然后将该子图转换为同构图。

In [34]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
})
sub_g = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                                   ('drug', 'interacts', 'gene')])
h_sub_g = dgl.to_homogeneous(sub_g)
h_sub_g

Graph(num_nodes=7, num_edges=4,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})